# Агент прикладного уровня для оценки качества сетевых сервисов (сетевая доступность, протокольная доступность, корректность авторизации, время отклика)

## По материалам: 
## https://habr.com/ru/company/proto/blog/532152/
## https://www.dynatrace.com/platform/digital-experience/
## https://www.dynatrace.com/platform/oneagent/
## https://betterstack.com/community/comparisons/javascript-application-monitoring-tools/



In [ ]:
#pip install aiohttp asyncssh

# nmap scanme.nmap.org
# Starting Nmap 7.80 ( https://nmap.org ) at 2023-02-12 10:16 UTC
# Nmap scan report for scanme.nmap.org (45.33.32.156)
# Host is up (0.19s latency).
# Other addresses for scanme.nmap.org (not scanned): 2600:3c01::f03c:91ff:fe18:bb2f
# Not shown: 993 closed ports
# PORT      STATE    SERVICE
# 22/tcp    open     ssh
# 53/tcp    open     domain
# 80/tcp    open     http
# 2323/tcp  filtered 3d-nfsd
# 5555/tcp  filtered freeciv
# 9929/tcp  open     nping-echo
# 31337/tcp open     Elite


In [5]:
import time
import asyncio
import asyncssh
import aiohttp
from http.client import HTTPConnection
from urllib.parse import urlparse
import ipywidgets

async def site_is_online_async(url, timeout,username,password):
    """Return True if the target URL is online.
    Raise an exception otherwise.
    """
    error = Exception("unknown scheme")
    
    parser = urlparse(url)
    host = parser.netloc or parser.path.split("/")[0]
    
    
    if parser.scheme in ("http", "https"):
        async with aiohttp.ClientSession() as session:
            try:
                await session.head(url, timeout=timeout,ssl=False)
                return True
            except asyncio.TimeoutError:
                error = Exception("timed out")
            except (asyncssh.Error) as e:
                error = e
                
    if parser.scheme in ("ssh"):
        if  parser.port == None : 
            port=22 
        else: 
            port=parser.port
            host=host.split(":")[0]

        #print(host,port)
        
        try:
            async with await asyncio.wait_for(asyncssh.connect(host=host,port=port,username=username, password=password, known_hosts=None), timeout=timeout) as conn:
                print((await conn.run('tty', term_type='ansi')).stdout, end='')
                return True
        except asyncio.TimeoutError:
                error = Exception("timed out")
        except (asyncssh.Error) as e:
                error = e
                
    if parser.scheme in ("postgre"):                
        if  parser.port == None : 
            port=5432
        else: 
            port=parser.port
            host=host.split(":")[0]
        
        coro = asyncio.open_connection(host, port)
        # execute the coroutine with a timeout
        try:
            # open the connection and wait for a moment
            _,writer = await asyncio.wait_for(coro, timeout)
            # close connection once opened
            writer.close()
            # indicate the connection can be opened
            return True
        except asyncio.TimeoutError:
                error = Exception("timed out")
        except (asyncssh.Error) as e:
                error = e
                
    if parser.scheme in ("kafka"):                
        if  parser.port == None : 
            port=9092
        else: 
            port=parser.port
            host=host.split(":")[0]
        
        coro = asyncio.open_connection(host, port)
        # execute the coroutine with a timeout
        try:
            # open the connection and wait for a moment
            _,writer = await asyncio.wait_for(coro, timeout)
            # close connection once opened
            writer.close()
            # indicate the connection can be opened
            return True
        except asyncio.TimeoutError:
                error = Exception("timed out")
        except (asyncssh.Error) as e:
                error = e
                
    if parser.scheme in ("raw"):                
        if  parser.port == None : 
            port=0
        else: 
            port=parser.port
            host=host.split(":")[0]
        
        coro = asyncio.open_connection(host, port)
        # execute the coroutine with a timeout
        try:
            # open the connection and wait for a moment
            _,writer = await asyncio.wait_for(coro, timeout)
            # close connection once opened
            writer.close()
            # indicate the connection can be opened
            return True
        except asyncio.TimeoutError:
                error = Exception("timed out")
        except (asyncssh.Error) as e:
                error = e                
                
    raise error
    
out1 = ipywidgets.Output(layout={'border': '1px solid black'})   

#@out1.capture()    
def display_check_result(latency_ns, url, error):
    """Display the result of a connectivity check."""
    #print(f'The status of "{url}" is:', end=" ")
    out1.append_stdout('\n')
    out1.append_stdout(f'The status of "{url}" is:')
    
    if (latency_ns):
        #print(f'"Online!" 👍, latency: {latency_ns*10e-9}s')
        out1.append_stdout(f'"Online!" 👍, latency: {latency_ns*10e-9}s')
    else:
        #print(f'"Offline?" 👎 \n  Error: "{error}"')
        out1.append_stdout(f'"Offline?" 👎 \n  Error: "{error}"')


async def _asynchronous_check(urls,timeout,username,password):
    async def _check(url,timeout,username,password):
        error = ""
        startTime = time.time_ns()
        try:
            await site_is_online_async(url,timeout,username,password)
            latency = time.time_ns() - startTime
        except Exception as e:
            latency = None
            error = str(e)
        display_check_result(latency, url,error)

    await asyncio.gather(*(_check(url,timeout,username,password) for url in urls))

def check(urls,timeout,username,password):
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:  # 'RuntimeError: There is no current event loop...'
        loop = None

    if loop and loop.is_running():
        #print('Async event loop already running. Adding coroutine to the event loop.')
        tsk = loop.create_task(_asynchronous_check(urls,timeout,username,password))
        # ^-- https://docs.python.org/3/library/asyncio-task.html#task-object
        # Optionally, a callback function can be executed when the coroutine completes
        #tsk.add_done_callback(
        #    lambda t: print(f'Task done with result={t.result()}  << return val of main()'))
    else:
        print('Starting new event loop')
        result = asyncio.run(_asynchronous_check(urls,timeout,username,password))
        

username = ipywidgets.Text(description='User:', placeholder='Введите здесь имя Ext-....')
password = ipywidgets.Password(description='Password:', placeholder='Введите здесь пароль')
sButton = ipywidgets.Button(description="Start tracing!")
cButton = ipywidgets.Button(description="Clear output")


def on_sButton_clicked(b):
    out1.append_stdout(f'\n{time.time()} - Start tracing!\n')
    check(timeout=15, urls=["http://scanme.nmap.org",     # HTTP via aiohttp 
                            "ssh://scanme.nmap.org",      # SSH via asyncssh 
                             "raw://scanme.nmap.org:22",   # RAW 22/tcp    open     ssh
                             "raw://scanme.nmap.org:53",   # RAW 53/tcp    open     domain
                             "raw://scanme.nmap.org:80",   # RAW 80/tcp    open     http
                             "raw://scanme.nmap.org:2323", # RAW 2323/tcp  filtered 3d-nfsd
                             "raw://scanme.nmap.org:5555", # RAW 5555/tcp  filtered freeciv
                             "raw://scanme.nmap.org:9929", # RAW 9929/tcp  open     nping-echo
                             "raw://scanme.nmap.org:31337" # RAW 31337/tcp open     Elite
                           ],
          username=username.value,password=password.value)        

def on_cButton_clicked(b):
    out1.clear_output()
                
sButton.on_click(on_sButton_clicked)
cButton.on_click(on_cButton_clicked)

display(ipywidgets.VBox([username,password]),ipywidgets.HBox([sButton,cButton]))
display(out1)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

## Пример выполнения однократной трассировки

1676530666.5861719 - Start tracing!

The status of "raw://scanme.nmap.org:53" is:"Online!" 👍, latency: 0.32056959s
The status of "raw://scanme.nmap.org:31337" is:"Online!" 👍, latency: 2.0859241600000002s
The status of "raw://scanme.nmap.org:9929" is:"Online!" 👍, latency: 2.16059916s
The status of "raw://scanme.nmap.org:80" is:"Online!" 👍, latency: 2.1979604200000002s
The status of "raw://scanme.nmap.org:22" is:"Online!" 👍, latency: 2.26143375s
The status of "http://scanme.nmap.org" is:"Online!" 👍, latency: 3.99740042s
The status of "ssh://scanme.nmap.org" is:"Offline?" 👎 
  Error: "Permission denied"
The status of "raw://scanme.nmap.org:2323" is:"Offline?" 👎 
  Error: "Multiple exceptions: [Errno 111] Connect call failed ('45.33.32.156', 2323), [Errno 99] Cannot assign requested address"
The status of "raw://scanme.nmap.org:5555" is:"Offline?" 👎 
  Error: "Multiple exceptions: [Errno 111] Connect call failed ('45.33.32.156', 5555), [Errno 99] Cannot assign requested address"